In [1]:
#importing required Libraries
import pandas as pd   #to create dataframe
import requests       #to send the request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML
import numpy as np  # to count the values (in our case)

#assigning the URL with variable name url
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
#url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start=101&ref_=adv_nxt'

#request allow you to send HTTP request
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

#creating an empty list, so that we can append the values
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
#Note: These three list are added recently, Not explained in the video
description = []
Director = []
Stars = []

#storing the meaningfull required data in the variable
movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})

#calling one by one using for loop
for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)
   
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)
   
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
   
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
   
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
    metascore.append(meta)
    #since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
    value = store.find_all('span', attrs = {'name': 'nv'})
   
    vote = value[0].text
    votes.append(vote)
   
    grosses = value[1].text if len(value) >1 else '*****'
    gross.append(grosses)
   
    # Description of the Movies -- Not explained in the Video, But you will figure it out.
    describe = store.find_all('p', class_ = 'text-muted')
    description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
    description.append(description_)
   
    #Cast Details -- Scraping Director name and Stars -- Not explained in Video
    cast = store.find("p", class_ = '')
    cast = cast.text.replace('\n', '').split('|')
    cast = [x.strip() for x in cast]
    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]
    Director.append(cast[0])
    Stars.append([x.strip() for x in cast[1].split(",")])
   
   
#creating a dataframe using pandas library
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description, "Director": Director, 'Star': Stars})

In [2]:
movie_DF.head()

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,1994,142,9.3,81,"2,661,415",$28.34M,Two imprisoned men bond over a number of years...,Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,844,408",$134.97M,The aging patriarch of an organized crime dyna...,Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,Kantara,2022,148,9.1,^^^^^^,"66,417",*****,It involves culture of Kambla and Bhootha Kola...,Rishab Shetty,"[Rishab Shetty, Sapthami Gowda, Kishore Kumar ..."
3,The Dark Knight,2008,152,9.0,84,"2,634,377",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
4,The Lord of the Rings: The Return of the King,2003,201,9.0,94,"1,835,145",$377.85M,Gandalf and Aragorn lead the World of Men agai...,Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O..."


In [4]:
movie_DF.describe()

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
count,100,100,100,100,100,100,100,100,100,100
unique,100,55,66,11,36,100,87,100,70,100
top,The Shawshank Redemption,2022,130,8.4,^^^^^^,"2,661,415",*****,Two imprisoned men bond over a number of years...,Christopher Nolan,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
freq,1,6,4,37,19,1,14,1,6,1
